# Custom Music Program

Project by: John (Jack) Kane, Sebastian Sepulveda, Bin Chen

We used Jupyter Notebook for this program

In [1]:
! pip install spotipy

import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
import requests
import random
import time
import webbrowser
import tkinter as tk
from tkinter import *
from tkinter import ttk

#Required credentials
#Spotify Developer credentials from developers.spotify.com
client_id = '2920ec1af7074a2f96e859b0246257f3'
client_secret = 'c986a95cc7f54a9a869ca9969eff9459'
redirect_uri = 'http://localhost:8888/callback/' 

#Authenticate with Spotify
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

custom_playlist = []

class main:
    def __init__(self, root):
        self.root = root
        self.root.title("Custom Music Playlist")
        self.root.geometry("1000x1000")
        self.root.configure(bg = "light blue")
        
        self.canvas = Canvas(self.root, width=900, height=250, bg="khaki1")
        self.canvas.pack()
        
        self.display = self.canvas.create_text(460, 125, text="", fill="black", font=('Helvetica 15 bold'))
        self.display2 = self.canvas.create_text(460, 215, text="", fill="black", font=('Helvetica 15 bold'))
        
        self.feedback = Label(root, text="", pady=10, font=("arial", 15, "bold"))
        
        self.intro_blurb()

    def intro_blurb(self):
        intro = f'''Welcome to our music program! This is a fun program that will build you a custom playlist based on YOUR interests
        and at no cost! You will be able to look into different music reccomendations based on year, genre, etc. and if you would 
        like add them to your custom playlist! Our hope is that this not only allows you to create a playlist to always enjoy
        jamming to, but that it may also introduce you to new music of different styles, cultures, and backgrounds that you
        may carry with you for many years. We hope you enjoy!'''
        self.canvas.itemconfig(self.display, text=intro.strip(), width=650, anchor = "center")

        self.next_page = ttk.Button(root, text="Next Page", command = self.song_of_the_day)
        self.next_page.pack()

    def song_of_the_day(self):
        self.next_page.destroy()
        
        song_of_day_intro = f'''Our song of the day is a current top hit, that has been taking over the music recently. Many
        people have been enjoying these songs, and we hope we can introduce you to one as well!'''

        self.canvas.itemconfig(self.display, text=song_of_day_intro, width=500)

        dic0 = {} 
        random_song = []
        
        for i in range(50):
            song_of_day_playlist = sp.playlist("37i9dQZF1DXcBWIGoYBM5M")
            song = song_of_day_playlist["tracks"]["items"][i]["track"]["name"]
            song_of_day_url = song_of_day_playlist["tracks"]["items"][i]['track']['preview_url']
            if song_of_day_url and not song_of_day_url.endswith('.wav'):
                song_of_day_url += '.wav'
                dic0[song] = song_of_day_url
                
        song_name0, song_preview0 = random.choice(list(dic0.items()))
        random_song.append(song_name0)

        song_of_add0 = f"Would you like to add {song_name0} to your playlist? Type YES to add it!"
        self.canvas.itemconfig(self.display2, text=song_of_add0, width=500)
        
        self.canvas.itemconfig(self.display, 
                          text=f"Today's song of the day is {random_song[0]}! We hope you give a listen and enjoy!",
                          width=500)
                        
        def callback(url):
            webbrowser.open_new_tab(url)

        if song_preview0 is None:
            self.dec_preview0 = Label(root, text="Preview: No preview was found for this song", pady = 10, font=("arial",10,"bold"))
            self.dec_preview0.pack()
        else:
            self.dec_preview0 = Label(root, text=f"Preview: {song_preview0}", pady = 10, font=("arial",10,"bold"))
            self.dec_preview0.pack()
            self.dec_preview0.bind("<Button-1>", lambda e:
            callback(f"{song_preview0}"))
        
        def add_playlist():
            custom_playlist.append(random_song[0])
            self.feedback["text"] = "Song added!"
            self.feedback.pack()
            self.yes_button.destroy()
            self.no_button.destroy()
            self.next_page1.pack()

        def no_add():
            self.yes_button.destroy()
            self.no_button.destroy()
            self.next_page1.pack()

        self.yes_button = Button(root, text="Yes", command=add_playlist, width=25, height=5)
        self.yes_button.pack()

        self.no_button = Button(root, text="No", command=no_add, width=25, height=5)
        self.no_button.pack()
        
        self.next_page1 = ttk.Button(root, text="Next Page", command = self.rec_song_by_artist)
        
    def rec_song_by_artist(self):
        self.dec_preview0.destroy()
        self.feedback.destroy()
        self.canvas.delete(self.display2)
        self.yes_button.destroy()
        self.no_button.destroy()
        self.next_page1.destroy()
        
        self.display2 = self.canvas.create_text(460, 215, text="", fill="black", font=('Helvetica 15 bold'))
        self.feedback = Label(root, text="", pady=10, font=("arial", 15, "bold"))
        
        artist_chosen = f"Please enter an artist you enjoy, and we will reccomend a song based on a similar artist:"
        self.canvas.itemconfig(self.display, text=artist_chosen)

        self.entry = Entry(root)
        self.entry.pack()

        reccomended_song = []

        def search_rec_artist():
            user_input = self.entry.get()

            results1 = sp.search(q=user_input, type='artist', limit=1)

            for artist in results1['artists']['items']:
                artist_id1 = artist['id']

            results = sp.recommendations(seed_artists=[artist_id1])
            rec_art = results["tracks"][3]['album']['artists'][0]['name']

            results2 = sp.search(q=rec_art, type='artist', limit=1)
            for k in results2['artists']['items']:
                artist_id2 = k['id']

            rec_songs = sp.artist_top_tracks(artist_id2)
            rec_song = rec_songs["tracks"][0]["name"]
            rec_art_preview = rec_songs["tracks"][0]["preview_url"]
            if rec_art_preview and not rec_art_preview.endswith('.wav'):
                rec_art_preview += '.wav'

            reccomended_song.append(rec_song)

            song_of_add1 = f"Would you like to add {rec_song}, by {rec_art} to your playlist?"
            self.canvas.itemconfig(self.display2, text=song_of_add1)
            
            def callback(url):
                webbrowser.open_new_tab(url)           
            
            if rec_art_preview is None:
                self.dec_preview1 = Label(root, text="Preview: No preview was found for this song", pady = 10, font=("arial",10,"bold"))
                self.dec_preview1.pack()
            else:
                self.dec_preview1 = Label(root, text=f"Preview: {rec_art_preview}", pady = 10, font=("arial",10,"bold"))
                self.dec_preview1.pack()
                self.dec_preview1.bind("<Button-1>", lambda e:
                callback(f"{rec_art_preview}"))

        self.search_button = Button(root, text="Search", command=search_rec_artist)
        self.search_button.pack()

        def add_playlist1():
            custom_playlist.append(reccomended_song[0])
            self.feedback["text"] = f"Song Added: {reccomended_song[0]}"
            self.feedback.pack()
            self.next_page2.pack()

        def no_add1():
            self.next_page2.pack()

        self.yes_button1 = Button(root, text="Yes", command=add_playlist1, width=25, height=5)
        self.yes_button1.pack()

        self.no_button1 = Button(root, text="No", command=no_add1, width=25, height=5)
        self.no_button1.pack()
        
        self.next_page2 = ttk.Button(root, text="Next Page", command = self.rec_song_by_genre)
        
    def rec_song_by_genre(self):
        self.dec_preview1.destroy()
        self.feedback.destroy()
        self.canvas.delete(self.display2)
        self.entry.destroy()
        self.search_button.destroy()
        self.yes_button1.destroy()
        self.no_button1.destroy()
        self.next_page2.destroy()
        
        genres_choices = ['alternative,','classical,','country,','edm,','folk,','indie-pop,','latino,','pop,','hip-hop,','rock,','work-out']
        
        self.display2 = self.canvas.create_text(460, 215, text="", fill="black", font=('Helvetica 15 bold'))
        self.feedback = Label(root, text="", pady=10, font=("arial", 15, "bold"))
        
        self.genre_list = Label(root, text = genres_choices, pady = 10, font=("arial",10,"bold"))
        self.genre_list.pack()
        
        genre_choice = "Please choose a genre from the list, and we will reccomend a song based on that genre"
        self.canvas.itemconfig(self.display, text=genre_choice)

        self.entry = Entry(root)
        self.entry.pack()

        genre_song = []

        def search_genre():
            user_choice = self.entry.get()

            rec1 = sp.recommendations(seed_genres=[user_choice], limit=10)
            rec_song_by_genre = rec1["tracks"][0]["name"]
            rec_art_by_genre = rec1["tracks"][0]["artists"][0]['name']
            genre_song.append(rec_song_by_genre)
            genre_preview = rec1["tracks"][0]["preview_url"]
            if genre_preview and not genre_preview.endswith('.wav'):
                genre_preview += '.wav'
                
            song_of_add2 = f"Would you like to add {rec_song_by_genre}, by {rec_art_by_genre} to your playlist?"
            self.canvas.itemconfig(self.display2, text=song_of_add2, width=500)
                        
            def callback(url):
                webbrowser.open_new_tab(url)

            if genre_preview is None:
                self.dec_preview2 = Label(root, text="Preview: No preview was found for this song", pady = 10, font=("arial",10,"bold"))
                self.dec_preview2.pack()
            else:
                self.dec_preview2 = Label(root, text=f"Preview: {genre_preview}", pady = 10, font=("arial",10,"bold"))
                self.dec_preview2.pack()
                self.dec_preview2.bind("<Button-1>", lambda e:
                callback(f"{genre_preview}"))

        self.search_button1 = Button(root, text="Search", command=search_genre)
        self.search_button1.pack()

        def add_playlist2():
            custom_playlist.append(genre_song[0])
            self.feedback["text"] = f"Song Added:{genre_song[0]}"
            self.feedback.pack()
            self.next_page3.pack()

        def no_add2():
            self.next_page3.pack()

        self.yes_button2 = Button(root, text="Yes", command=add_playlist2, width=25, height=5)
        self.yes_button2.pack()

        self.no_button2 = Button(root, text="No", command=no_add2, width=25, height=5)
        self.no_button2.pack()

        self.next_page3 = ttk.Button(root, text="Next Page", command = self.rec_song_by_album)
        
    def rec_song_by_album(self):
        self.dec_preview2.destroy()
        self.genre_list.destroy()
        self.feedback.destroy()
        self.canvas.delete(self.display2)
        self.entry.destroy()
        self.search_button1.destroy()
        self.yes_button2.destroy()
        self.no_button2.destroy()
        self.next_page3.destroy()
        
        self.display2 = self.canvas.create_text(460, 215, text="", fill="black", font=('Helvetica 15 bold'))
        self.feedback = Label(root, text="", pady=10, font=("arial", 15, "bold"))
        
        album_chosen = "All playlists need some familiarity, so please enter your favorite album, and we will choose a song from that album:"
        self.canvas.itemconfig(self.display, text=album_chosen)

        self.entry = Entry(root)
        self.entry.pack()

        random_album_song = []

        def search_album():
            album_choice = self.entry.get()

            results2 = sp.search(q=album_choice, type='album', limit=1)
            for album in results2['albums']['items']:
                album_id1 = album['id']

            alb_info = sp.album(album_id1)
            songs_from_album = alb_info['tracks']['items']

            dic3 = {}

            for i in songs_from_album:
                alb_song = i['name']
                alb_preview = i['preview_url']
                if alb_preview and not alb_preview.endswith('.wav'):
                    alb_preview += '.wav'
                dic3[alb_song] = alb_preview
                
            song_name3, song_preview3 = random.choice(list(dic3.items()))
            random_album_song.append(song_name3)

            song_of_add3 = f"Would you like to add {song_name3} to your playlist? Type YES to add it!"
            self.canvas.itemconfig(self.display2, text=song_of_add3, width=500)
                        
            def callback(url):
                webbrowser.open_new_tab(url)

            if song_preview3 is None:
                self.dec_preview3 = Label(root, text="Preview: No preview was found for this song", pady = 10, font=("arial",10,"bold"))
                self.dec_preview3.pack()
            else:
                self.dec_preview3 = Label(root, text=f"Preview: {song_preview3}", pady = 10, font=("arial",10,"bold"))
                self.dec_preview3.pack()
                self.dec_preview3.bind("<Button-1>", lambda e:
                callback(f"{song_preview3}"))

        self.search_button2 = Button(root, text="Search", command=search_album)
        self.search_button2.pack()

        def add_playlist3():
            custom_playlist.append(random_album_song[0])
            self.feedback["text"] = f"Song Added:{random_album_song[0]}"
            self.feedback.pack()
            self.next_page4.pack()

        def no_add3():
            self.next_page4.pack()

        self.yes_button3 = Button(root, text="Yes", command=add_playlist3, width=25, height=5)
        self.yes_button3.pack()

        self.no_button3 = Button(root, text="No", command=no_add3, width=25, height=5)
        self.no_button3.pack()

        self.next_page4 = ttk.Button(root, text="Next Page", command = self.rec_song_by_decade)

    def rec_song_by_decade(self):
        self.dec_preview3.destroy()
        self.feedback.destroy()
        self.canvas.delete(self.display2)
        self.entry.destroy()
        self.search_button2.destroy()
        self.yes_button3.destroy()
        self.no_button3.destroy()
        self.next_page4.destroy()
        
        self.display2 = self.canvas.create_text(460, 215, text="", fill="black", font=('Helvetica 15 bold'))
        self.feedback = Label(root, text="", pady=10, font=("arial", 15, "bold"))
        
        decade = "Please enter your FAVORITE decade for music, and we will reccomend one of the top hits from that decade:" 
        self.canvas.itemconfig(self.display, text=decade)

        self.entry = Entry(root)
        self.entry.pack()

        decades_throwback = []

        def search_decade():
            decade_chosen = self.entry.get()

            dec_results = sp.search(q = decade_chosen, type= 'playlist', limit= 1)
            playlist_id = dec_results['playlists']['items'][0]['id']

            dec_results1 = sp.playlist(playlist_id)
            dec_results2 = dec_results1['tracks']['items']

            dic4 = {}

            for i in dec_results2:
                dec_song = i['track']['name']
                dec_preview = i['track']['preview_url']
                if dec_preview and not dec_preview.endswith('.wav'):
                    dec_preview += '.wav'
                dic4[dec_song] = dec_preview
                
            song_name4, song_preview4 = random.choice(list(dic4.items()))
            decades_throwback.append(song_name4)

            song_of_add4 = f"Would you like to add {song_name4} to your playlist? Type YES to add it!"
            self.canvas.itemconfig(self.display2, text=song_of_add4, width=500)
                        
            def callback(url):
                webbrowser.open_new_tab(url)

            if song_preview4 is None:
                self.dec_preview4 = Label(root, text="Preview: No preview was found for this song", pady = 10, font=("arial",10,"bold"))
                self.dec_preview4.pack()
            else:
                self.dec_preview4 = Label(root, text=f"Preview: {song_preview4}", pady = 10, font=("arial",10,"bold"))
                self.dec_preview4.pack()
                self.dec_preview4.bind("<Button-1>", lambda e:
                callback(f"{song_preview4}"))

        self.search_button3 = Button(root, text="Search", command=search_decade)
        self.search_button3.pack()

        def add_playlist4():
            custom_playlist.append(decades_throwback[0])
            self.feedback["text"] = f"Song Added:{decades_throwback[0]}"
            self.feedback.pack()
            self.next_page5.pack()

        def no_add4():
            self.next_page5.pack()

        self.yes_button = Button(root, text="Yes", command=add_playlist4, width=25, height=5)
        self.yes_button.pack()

        self.no_button = Button(root, text="No", command=no_add4, width=25, height=5)
        self.no_button.pack()

        self.next_page5 = ttk.Button(root, text="View your playlist", command = self.playlist)
        
    def playlist(self):
        self.dec_preview4.destroy()
        self.feedback.destroy()
        self.canvas.delete(self.display2)
        self.entry.destroy()
        self.search_button3.destroy()
        self.yes_button.destroy()
        self.no_button.destroy()
        self.next_page5.destroy()
        
        self.canvas.itemconfig(self.display, text="We hope you enjoy your custom playlist!")
        
        self.canvas.itemconfig(self.display2, text="Here is your custom playlist:")
        
        for i in custom_playlist:
            self.label = Label(root, text=i, pady=10, font=("arial", 15, "bold"))
            self.label.pack()
        

root = Tk()
main(root)
root.mainloop()